# Code Verification Notebook

This notebook is used to provide a means of validating the accuracy of the underlying logic used to solve the Schrödinger equation. This notebook uses a Harmonic Oscillator potential combined with a Harmonic Oscillator basis set to determine accuracy. Since the basis set is the analytical solution to the Schrödinger equation given the potential, the eigen-vectors should consist of a single 1 value, while other values in the vector are 0, since only a single basis function can be used to describe the solution. </br></br>
In addition, the energy levels of the quantum Harmonic Oscillator are known and can be directly calculated to compare against the eigen-values produced by the code, further validating its functionality for more complicated systems. </br></br>
The theory presented in this notebook is based of of the following website: <a href="http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/hosc.html">HyperPhysics.</a>

In [1]:
#All imports go here

import sys
if("win" in sys.platform):
    sys.path.append("..\\")
else: 
    sys.path.append("../")
    
from how import how
from compChemGlobal import * 
from schrodinger import schrod
from basisSet import basisSet
from howPotential import howPotential
from nistScraper import getDiatomicConstants
from operators import TOperator, VOperator

In [2]:
#Declare all global variables/functions here
testingDiatomics = ["H2", "NO"]
basisSize = 20
zeroCutOff = 25e-4

def energy(w, n): 
    return (n + 0.5) * w 

###################################################################################

def message(msg, error):
    print( "Warning!" if error else "Pass: ", msg)

###################################################################################

#Returns bool specifying if test was a pass or a fail
def checkEigenVectors(eigenVectors):
    
    for index, vector in enumerate(eigenVectors):
        vector = abs(vector)
        notFound1 = True
        for value in vector: 
            #check if 0 or 1
            if(value > zeroCutOff):
                if(notFound1):
                    if(value - 1 > zeroCutOff):
                        message("Eigen-Vector " +  str(index) + " Failed, value greater than 1.", True)
                        return False
                    notFound1 = False
                else: 
                    message("Eigen-Vector " + str(index) + " Failed, multiple values are 1.", True)
                    return False
                
        if(notFound1):
            message("Eigen-Vector " + str(index) + " Failed, no value of 1 found.", True)
            return False
        else: 
            message("Eigen-Vector " + str(index) + " Passed.", False)
        
    return True

###################################################################################

#Returns tuple of error percentages and true or false for test status
def checkEigenValues(w, eigenValues):
    errorValues = []
    for n, eigenValue in enumerate(eigenValues):
        correctValue = energy(w, n)
        errorValues.append( abs(correctValue - eigenValue) / eigenValue * 100 )
        
        if(errorValues[-1] > 0.01):
            message("Eigen-Value " + str(n) + " Failed.", True)
            return (errorValues, False)
    
    message("Eigen-Values Passed.\nHighest Error Percentage: " + str(max(errorValues)) + "%", False)
    return (errorValues, True)

In [3]:
reports = {}

for diatomic in testingDiatomics: 
    
    #Run System
    dc = getDiatomicConstants(diatomic)
    pes = howPotential(dc)
    basis = basisSet(dc, how, basisSize)
    H = TOperator(basis) + VOperator(basis, pes)
    sol = schrod(H, basis)
    
    reports[diatomic] = {}
    print("***********", diatomic, "Test Report", "***********\n")
    
    reports[diatomic]["eigen-vectors"] = (sol.eigenVectors, checkEigenVectors(sol.eigenVectors))
    print()
    
    eigenValuesTest = [sol.eigenValues]
    eigenValuesTest.extend(checkEigenValues(dc["w"], sol.eigenValues))
    reports[diatomic]["eigen-values"] = tuple(eigenValuesTest)
    
    sol.graph()
    
    print("\n***************************************\n")

*********** H2 Test Report ***********

Pass:  Eigen-Vector 0 Passed.
Pass:  Eigen-Vector 1 Passed.
Pass:  Eigen-Vector 2 Passed.
Pass:  Eigen-Vector 3 Passed.
Pass:  Eigen-Vector 4 Passed.
Pass:  Eigen-Vector 5 Passed.
Pass:  Eigen-Vector 6 Passed.
Pass:  Eigen-Vector 7 Passed.
Pass:  Eigen-Vector 8 Passed.
Pass:  Eigen-Vector 9 Passed.
Pass:  Eigen-Vector 10 Passed.
Pass:  Eigen-Vector 11 Passed.
Pass:  Eigen-Vector 12 Passed.
Pass:  Eigen-Vector 13 Passed.
Pass:  Eigen-Vector 14 Passed.
Pass:  Eigen-Vector 15 Passed.
Pass:  Eigen-Vector 16 Passed.
Pass:  Eigen-Vector 17 Passed.
Pass:  Eigen-Vector 18 Passed.
Pass:  Eigen-Vector 19 Passed.

Pass:  Eigen-Values Passed.
Highest Error Percentage: 0.002350719585725138%


    'data': [{'fill': 'tozerox',
              'hoverlabel': {'font': {'size': 1…


***************************************

*********** NO Test Report ***********

Pass:  Eigen-Vector 0 Passed.
Pass:  Eigen-Vector 1 Passed.
Pass:  Eigen-Vector 2 Passed.
Pass:  Eigen-Vector 3 Passed.
Pass:  Eigen-Vector 4 Passed.
Pass:  Eigen-Vector 5 Passed.
Pass:  Eigen-Vector 6 Passed.
Pass:  Eigen-Vector 7 Passed.
Pass:  Eigen-Vector 8 Passed.
Pass:  Eigen-Vector 9 Passed.
Pass:  Eigen-Vector 10 Passed.
Pass:  Eigen-Vector 11 Passed.
Pass:  Eigen-Vector 12 Passed.
Pass:  Eigen-Vector 13 Passed.
Pass:  Eigen-Vector 14 Passed.
Pass:  Eigen-Vector 15 Passed.
Pass:  Eigen-Vector 16 Passed.
Pass:  Eigen-Vector 17 Passed.
Pass:  Eigen-Vector 18 Passed.
Pass:  Eigen-Vector 19 Passed.

Pass:  Eigen-Values Passed.
Highest Error Percentage: 0.0023507195857543047%


    'data': [{'fill': 'tozerox',
              'hoverlabel': {'font': {'size': 1…


***************************************

